In [2]:
import socket
import logging
import os

# Set up logging without timestamps
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# Define the directory where you want to save the file
SAVE_DIRECTORY = "C:/Users/MPC/Downloads/saratest"  # Save in the current working directory, change as needed

# Ensure the directory exists
if not os.path.exists(SAVE_DIRECTORY):
    os.makedirs(SAVE_DIRECTORY)

# Function to get a filename with a timestamp
def get_filename():
    return os.path.join(SAVE_DIRECTORY, "hl7_messages.txt")

def extract_bed_number(hl7_message):
    """
    Extracts the bed number from an HL7 message.
    
    Args:
        hl7_message (str): The HL7 message string.
        
    Returns:
        str: The bed number extracted from the message, or None if not found.
    """
    try:
        segments = hl7_message.split('\r')
        for segment in segments:
            if segment.startswith('PID'):
                fields = segment.split('|')
                bed_number = fields[4] if len(fields) > 4 else None
                return bed_number
    except Exception as e:
        logging.error(f"Error extracting bed number: {e}")
    return None

def save_hl7_message_to_file(hl7_message, filename):
    """
    Saves the HL7 message to a text file.
    
    Args:
        hl7_message (str): The HL7 message string to save.
        filename (str): The name of the file to save the message.
    """
    with open(filename, "a") as f:  # Open the file in append mode
        f.write(hl7_message + "\n")  # Save the HL7 message with a newline separator

def process_hl7_message(hl7_message, filename):
    """
    Processes a single HL7 message and saves it to a file.
    
    Args:
        hl7_message (str): The HL7 message string.
        filename (str): The name of the file to save the message.
    """
    if hl7_message.strip():
        logging.info(f"Received HL7 message: {hl7_message}")
        save_hl7_message_to_file(hl7_message, filename)  # Save the message to the text file

def start_server(host='175.31.5.50', port=8000):
    """
    Starts a TCP server that listens for incoming connections and processes HL7 messages.
    
    Args:
        host (str): The hostname or IP address to bind the server to.
        port (int): The port number to bind the server to.
    """
    filename = get_filename()  # Get the filename for the current run
    logging.info(f"Saving messages to {filename}")
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        try:
            server_socket.bind((host, port))
            server_socket.listen(5)
            logging.info(f"Server listening on {host}:{port}...")
        except socket.error as e:
            logging.error(f"Socket error: {e}")
            return

        while True:
            conn, addr = server_socket.accept()
            logging.info(f"Connected by {addr}")
            handle_connection(conn, filename)
            logging.info(f"Connection with {addr} closed.")

def handle_connection(conn, filename):
    """
    Handles incoming connection and processes received HL7 messages.
    
    Args:
        conn (socket): The connection object.
        filename (str): The name of the file to save the messages.
    """
    with conn:
        buffer = b''  # Use a bytes buffer
        while True:
            try:
                data = conn.recv(1024)
                if not data:
                    break  # Exit the loop if no more data
                
                buffer += data
                decoded_data = buffer.decode('utf-8', errors='ignore')
                messages = decoded_data.split('\r')

                # Process all complete messages
                for message in messages[:-1]:
                    process_hl7_message(message, filename)

                # Keep the last potentially incomplete message in buffer
                buffer = messages[-1].encode('utf-8')

            except UnicodeDecodeError as e:
                logging.error(f"Unicode decoding error: {e}")
                continue

        # Process any remaining data in the buffer after connection is closed
        if buffer:
            decoded_data = buffer.decode('utf-8', errors='ignore')
            messages = decoded_data.split('\r')
            for message in messages:
                process_hl7_message(message, filename)

if __name__ == "__main__":
    start_server()


INFO - Saving messages to C:/Users/MPC/Downloads/saratest\hl7_messages.txt
INFO - Server listening on 175.31.5.50:8000...
